# Hyperspectral Material Classification - Training (Google Colab)

Optimized for Google Colab Pro+ with A100 GPU

In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install spectral scikit-learn

In [ ]:
!git clone https://github.com/PlugNawapong/hsi-deeplearning.git
%cd hsi-deeplearning

In [ ]:
# See TRAINING_GUIDE.md for full training instructions
# Modify the data paths below to point to your Google Drive

CONFIG = {
    'data_path': '/content/drive/MyDrive/training_dataset_normalized',
    'labels_path': '/content/drive/MyDrive/Ground_Truth/labels.json',
    'output_dir': '/content/drive/MyDrive/outputs/colab',
    'model_type': 'SpectralCNN1D',
    'batch_size': 256,
    'num_epochs': 100,
    'learning_rate': 0.001
}